In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as ns
import xml.etree.ElementTree as ET
import time
import re

## Reding xml data into pandas dataframe

In [2]:
def read_xml(xml_filename):
    xtree = ET.parse(xml_filename)
    xroot = xtree.getroot() 
    
    # getting the columns
    df_cols = []
    for x in xroot:
        for attr in x.attrib:
            if attr not in df_cols:
                df_cols.append(attr)
    
    # getting the records
    rows = []
    for x in xroot:
        row = dict()
        for col in df_cols:
            if col in x.attrib:
                row[col] = x.attrib[col]
            else:
                row[col] = None
        rows.append(row)
        
    out_df = pd.DataFrame(rows, columns = df_cols)
    return out_df

In [3]:
%%time
badges = read_xml("data/Badges.xml")
print("badges:", badges.shape)
comments = read_xml("data/Comments.xml")
print("comments:", comments.shape)
post_history = read_xml("data/PostHistory.xml")
print("post_history:", post_history.shape)
post_links = read_xml("data/PostLinks.xml")
print("post_links:", post_links.shape)
posts = read_xml("data/Posts.xml")
print("posts:", posts.shape)
tags = read_xml("data/Tags.xml")
print("tags:", tags.shape)
users = read_xml("data/Users.xml")
print("users:", users.shape)
votes = read_xml("data/Votes.xml")
print("votes:", votes.shape)

badges: (78180, 6)
comments: (41722, 7)
post_history: (121522, 9)
post_links: (1653, 5)
posts: (39929, 21)
tags: (489, 5)
users: (66951, 13)
votes: (125650, 6)
Wall time: 5.37 s


In [4]:
start_time = time.time()

## Generating insights

### 1. Working with the Posts table

In [5]:
posts['CreationDate'] = pd.to_datetime(posts['CreationDate'], format="%Y-%m-%dT%H:%M:%S")
posts['CreationYear'] = posts['CreationDate'].dt.year
posts.head()

,Id,PostTypeId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,...,FavoriteCount,ClosedDate,AcceptedAnswerId,LastEditorUserId,LastEditDate,ParentId,CommunityOwnedDate,LastEditorDisplayName,OwnerDisplayName,CreationYear
0,5,1,2014-05-13 23:58:30.457,8,604,<p>I've always been interested in machine lear...,5,2014-05-14T00:36:31.077,How can I do simple machine learning without h...,<machine-learning>,...,1,2014-05-14T14:40:25.950,None,None,None,None,None,None,None,2014
1,7,1,2014-05-14 00:11:06.457,4,426,"<p>As a researcher and instructor, I'm looking...",36,2014-05-16T13:45:00.237,What open-source books (or other materials) pr...,<education><open-source>,...,1,2014-05-14T08:40:54.950,10,97,2014-05-16T13:45:00.237,None,None,None,None,2014
2,9,2,2014-05-14 00:36:31.077,5,None,"<p>Not sure if this fits the scope of this SE,...",51,2014-05-14T00:36:31.077,None,None,...,None,None,None,None,None,5,None,None,None,2014
3,10,2,2014-05-14 00:53:43.273,12,None,"<p>One book that's freely available is ""The El...",22,2014-05-14T00:53:43.273,None,None,...,None,None,None,None,None,7,None,None,None,2014
4,14,1,2014-05-14 01:25:59.677,23,1495,<p>I am sure data science as will be discussed...,66,2014-06-20T17:36:05.023,Is Data Science the Same as Data Mining?,<data-mining><definitions>,...,6,None,29,322,2014-06-17T16:17:20.473,None,None,None,None,2014


In [6]:
yearTags = posts[['CreationYear', 'Tags']]
print(yearTags.shape)
yearTags = yearTags.dropna() # removing rows that has null tags
print(yearTags.shape)
yearTags.Tags = yearTags.Tags.str.replace('<', '').str.replace('>', ' ').str.split()
yearTags.head()

(39929, 2)
(18513, 2)


,CreationYear,Tags
0,2014,[machine-learning]
1,2014,"[education, open-source]"
4,2014,"[data-mining, definitions]"
5,2014,[databases]
6,2014,"[machine-learning, bigdata, libsvm]"


In [7]:
%%time
popularTagsInYear = dict()

for index, row in yearTags.iterrows():
    if row['CreationYear'] not in popularTagsInYear:
        popularTagsInYear[row['CreationYear']] = dict()
    curDict = popularTagsInYear[row['CreationYear']]
    for tag in row['Tags']:
            if tag not in curDict:
                curDict[tag] = 0
            curDict[tag] += 1

rows_list = []
for yearItem in popularTagsInYear.items():
    for key, value in sorted(yearItem[1].items(), key=lambda item: item[1], reverse=True)[:5]:
        rows_list.append([yearItem[0], key, value])
        #print("%s: %s" % (key, value))
popularTagsInYearDf = pd.DataFrame(rows_list, columns=['year','tag','count'])
popularTagsInYearDf.head()

Wall time: 1.48 s


,year,tag,count
0,2014,machine-learning,171
1,2014,data-mining,74
2,2014,classification,73
3,2014,bigdata,61
4,2014,r,54


In [8]:
print("It took %s seconds to process" % (time.time() - start_time))

It took 1.6653642654418945 seconds to process
